In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@markdown # Install/Update roop
from IPython.display import clear_output
from IPython.utils import capture
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

if not os.path.exists("/content/"):
    print('[1;31mGdrive not connected, using colab storage ...')
    time.sleep(4)
    !mkdir -p /content/
with capture.capture_output() as cap:
    def inf(msg, style, wdth):
        inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
        display(inf)

    %mkdir -p /content/gdrive/MyDrive
    %cd /content/gdrive/MyDrive
    !git clone https://github.com/s0md3v/roop.git

with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/roop/
    !git reset --hard
    time.sleep(1)
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Requirements
import hashlib


# Based on https://stackoverflow.com/questions/22058048/hashing-a-file-in-python
def hash_mistmatch(file_, hash):
    h = hashlib.sha256()
    with open(file_, 'rb') as file:
        while True:
        # Reading is buffered, so we can read smaller chunks.
            chunk = file.read(h.block_size)
            if not chunk:
                break
            h.update(chunk)
        retry = h.hexdigest() != hash
        if retry:
            inf(f'\u2718 Mismatch in hash of {file_}','warning', '200px')
        else:
            inf(f'\u2714 Verified integrity of {file_} successfully','info', '200px')
        return retry


print('[1;32mInstalling requirements...')


# 14ca261f48bdd88c1eecba96a761bd1579b523adae1b749b0a4ffd8b7ed8babe  /root/.opennsfw2/open_nsfw_weights.h5
# 80ffe37d8a5940d59a7384c201a2a38d4741f2f3c51eef46ebb28218a7b0ca2f  /root/.insightface/models/buffalo_l.zip

with capture.capture_output() as cap:
    %cd /content/gdrive/MyDrive/roop/
    !wget -O "inswapper_128.onnx" https://civitai.com/api/download/models/85159
    !pip install onnxruntime-gpu
    !pip install -r requirements.txt
    !mkdir /content/gdrive/MyDrive/roop_output

    # Suggested by @phineas-pta
retry_count = 0
while retry_count < 3:
    with capture.capture_output() as cap:
        !wget -P ~/.opennsfw2/weights https://github.com/bhky/opennsfw2/releases/download/v0.1.0/open_nsfw_weights.h5
    if not hash_mistmatch('/root/.opennsfw2/weights/open_nsfw_weights.h5', '14ca261f48bdd88c1eecba96a761bd1579b523adae1b749b0a4ffd8b7ed8babe'):
        retry_count = 0
        break
    retry_count += 1
while retry_count < 3:
    with capture.capture_output() as cap:
        !wget -P ~/.insightface/models https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip
    if not hash_mistmatch('/root/.insightface/models/buffalo_l.zip', '80ffe37d8a5940d59a7384c201a2a38d4741f2f3c51eef46ebb28218a7b0ca2f'):
        retry_count = 0
        break
    retry_count += 1

with capture.capture_output() as cap:
    !cd ~/.insightface/models && unzip -o buffalo_l.zip -d buffalo_l


# clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown # Download example files

print('[1;32mDownload example files...')

with capture.capture_output() as cap:
    !mkdir /content/gdrive/MyDrive/example
    %cd /content/gdrive/MyDrive/example
    !wget -O "img.jpg" https://pinkmirror.com/getImagePerson.ashx?id=16051&&photo_type=photo
    !wget -O "vid.mp4" https://github.com/ai-forever/ghost/raw/main/examples/videos/nggyup.mp4

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@markdown #**Select source image and target video**

#@markdown Choose source image as a photo, preferable a selfie of a person


source_path = '/content/gdrive/MyDrive/example/img.jpg' #@param {type:"string"}
target_path = '/content/gdrive/MyDrive/example/vid.mp4' #@param {type:"string"}
path_to_output = '/content/gdrive/MyDrive/roop_output/out_vid.mp4' #@param {type:"string"}
keep_fps = True #@param {type:"boolean"}
keep_frames = False #@param {type:"boolean"}

kframes = "--keep-frames" if keep_frames else ""
kfps = "--keep-fps" if keep_fps else ""

OUT_VIDEO_NAME = path_to_output

!python /content/gdrive/MyDrive/roop/run.py -f $source_path -t $target_path -o $path_to_output $kfps $kframes --gpu-vendor nvidia


In [ ]:
#@markdown #**Visualize Video Swap**

from IPython.display import HTML
from base64 import b64encode

video_file = open(OUT_VIDEO_NAME, "r+b").read()
video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

HTML(f"""<video width={800} controls><source src="{video_url}"></video>""")